In [ ]:
%load_ext autoreload
%autoreload 2

import lib.Toolbox as tb
import lib.TimeKeeper as tk
import pandas as pd
import matplotlib.pyplot as plt

# What Needs Doing
Here's the painful truth: CVNA may not be a good stock to trade forever. We need to be able to identify when a stock is good to trade and then only trade on it during that time. That's going to require some pretty big, weird things, that honestly make us sort of uncomfortable because they involve more digging into this topic and we're honestly pretty tired of it all, but here goes:

1. A way to monitor what people are saying around the market on a given day, aka, some sort of bot that looks at the web to read for sentiment.
2. Better language to describe what a market is doing and understand where it's going (mastering trend, aka, if we see a trend forming, we act on that in addition to our bands)

What exactly are we doing here? We need a script that:
1. Takes a symbol name and creates an archive of it
2. Does some sort of process to determine what trading strategy/config/selection strategy would be best for that archive (this will need to be able to take a date so we can update it for a given day)
3. Store that information in a manner that will allow other processes to utilize it when making decisions about what to trade with on a given day and, during that day, what to trade.

What do we do today?
1. Update the archive functions to also collect month data and five minute data. Five minute data should be sufficient for broadly predicting behavior.
2. Create functions that broadly measure the effectiveness of bollinger bands of MACD for a set of data
3. Research terms that we don't know much about
4. Research other methods of predicting market viability


At some point I'd like to see if we can actually get anything out of the OBV for day selection, but I think that idea is really too half baked to implement effectively right now

# New Terms
Choppy Market: Buyers and sellers are in balance, there's no clear trend.

Swing Highs and Swing Lows: Basically this means that the highs keep getting higher and the lows keep getting lower. If the last min was less than the min before, we may be swinging low, if the last high was higher than the previous high and the last low was higher than the previous low then our trend is likely upward, if it's the reverse then it's likely lower, if there's no clear pattern then we are probably in a choppy market.

Bollinger bands thrive most in choppy markets, as crazy as that sounds.

Some Initial Thoughts:
Make a "choppiness" function that determines whether a symbol is in choppy conditions. This will get the relative highs and lows and see if they are balanced or clearly trending. Then we test whether or not our bollinger strategy works in that circumstance using CVNA.

In [ ]:
data = tb.get_archive('cvna', '5m')

# Assess Bollinger Potential
What are my basic assumptions? My basic assumption here is that accuracy of bollinger bands over five minute data will predict effectiveness of bollinger bands over 1 minute data. I think that's a fair assessment, but we can test it by checking it with cvna, as we have both five minute and one minute data for it going back a good while.

But what does this actually mean? It means, if we have a bollinger event, check if it's met by a corresponding bollinger event.

In [ ]:
def show_bollinger(data, bollinger, dev, day):
    bollinger = bollinger[day:]
    data = data[day:]
    plt.title(dev)
    plt.plot(data.index, data.open, lw=.5)
    plt.plot(bollinger.index, bollinger.lowband, color='green', lw=.5)
    plt.plot(bollinger.index, bollinger.highband, color='red', lw=.5)
    plt.plot(bollinger.index, bollinger.moving_average, color='yellow', lw=.5)
    plt.show()

def find_best_bollinger_fit(index:pd.DatetimeIndex, data:pd.DataFrame):
    for bol_dev in [1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2]:
        dayta = data[:index]
        bollinger_bands = tb.get_bollinger_bands(dayta.open, num_stds=(bol_dev, 0, -bol_dev))
        show_bollinger(dayta, bollinger_bands, bol_dev, tk.today())

def get_best_bollinger(data:pd.DataFrame):
    time = tk.to_time(1,12,9,46)
    find_best_bollinger_fit(time, data)
    print(archive.loc[time])

get_best_bollinger(archive)

In [ ]:
# Assess Bollinger

# Rectangle Assessment

# Assess Trend

In [ ]:
mo_data = tb.get_archive('cvna', '1d')
data = tb.get_archive('cvna', '1m')

def assess_trend(index:pd.DatetimeIndex, data:pd.DataFrame, buffer:int = 3):
    '''Determines whether the given day is likely to be in an upswing, and downward swing, or a choppy market'''
    i = tb.get_i(index, data)
    if i < buffer: print('Must have at least',buffer,'days to derive trend'); return
    delta = data.iloc[i-buffer:i].diff()
    high_trend, low_trend = delta.high.sum() > 0, delta.low.sum() > 0

    if high_trend and low_trend: return 'UP'
    elif not high_trend and not low_trend: return 'DOWN'
    else: return 'CHOP'

decisions = []
subset = mo_data[-60:]
for day in subset.index:
    decisions.append([day,assess_trend(day, mo_data)])

plt.plot(subset.index, subset.open, lw=.5)
for dec in decisions:
    if dec[1] == 'UP': plt.axvline(dec[0], color='green',lw=.5)
    elif dec[1] == 'DOWN': plt.axvline(dec[0], color='red',lw=.5)
    elif dec[1] == 'CHOP': plt.axvline(dec[0], color='yellow',lw=.5)
plt.show()

for dec in decisions:
    day_subset = tk.get_workday(data, dec[0])
    plt.plot(day_subset.index, day_subset.open, lw=.5)
    plt.title(dec[0])
    for index in day_subset.index:
        if dec[1] == 'UP': plt.axvline(index, color='green',lw=.5, alpha=.15)
        elif dec[1] == 'DOWN': plt.axvline(index, color='red',lw=.5, alpha=.15)
        elif dec[1] == 'CHOP': plt.axvline(index, color='yellow',lw=.5, alpha=.15)
    plt.show()

# Simple Candle Stick Analysis

In [ ]:
data = tb.get_data('amzn', '1m', '1d')

In [ ]:
ups, downs = [], []

def predict_change(data:pd.DataFrame, i:int, lookback:int=3):
    entries = data[i-lookback:i]
    clop = entries.close - entries.open
    dclop = clop.diff()

    sway = 0
    if dclop[-1] > 0: sway += 1
    else: sway -= 1
    # if clo > op:
        # if op - lo < clo - op: print('next is going up'); ups.append(index)
        # elif op - lo > op - clo: print('next is going down'); downs.append(index)
    if sway > 0: ups.append(data.index[i])
    else: downs.append(data.index[i])

for i in range(len(data.index))[-30:-1]:
    print(data.index[i])
    predict_change(data, i)

df = data[-30:-1]
plt.plot(df.index, df.open, lw=.5)
for up in ups:
    print(up)
    plt.axvline(up, color='green', lw=.5)
for down in downs:
    plt.axvline(down, color='red', lw=.5)
plt.show()



In [ ]:
ups, downs = [], []

def predict_change(data:pd.DataFrame, i:int, lookback:int=1):
    entries = data[i-lookback:i]
    last = entries.iloc[-1]

    # We fell during this entry
    if last.open > last.close:
        upward_pressure = last.high - last.open
        downward_pressure = last.close - last.low
        candle = last.open - last.close
    # We rose
    else:
        upward_pressure = last.high - last.close
        downward_pressure = last.open - last.low
        candle = last.close - last.open

    if upward_pressure > downward_pressure: ups.append(data.index[i]); return True
    elif upward_pressure < downward_pressure: downs.append(data.index[i]); return False

success, total = 0, 0
failures = []
for i in range(len(data.index))[-120:-1]:
    going_up = predict_change(data, i)
    total += 1
    if going_up and data.iloc[i].open < data.iloc[i].high: success += 1
    elif not going_up and data.iloc[i].open > data.iloc[i].low: success += 1
    else: print('FAILED AT',data.index[i]); failures.append(data.index[i])

print(success/total)

df = data[-120:-1]
plt.figure(dpi=200)
plt.plot(df.index, df.open, lw=.5)
for up in ups:
    print(up)
    plt.axvline(up, color='green', lw=.5)
for down in downs:
    plt.axvline(down, color='red', lw=.5)
for fails in failures:
    plt.axvline(fails, color='violet', lw=.75)#, alpha=.25)

plt.show()


In [ ]:
'''
THEORY CRAFTING
We can call the difference between the high and (open close) as the "UPWARD PRESSURE"
and the difference between the low and (open close) as the "DOWNWARD PRESSURE"

If something has strong downward pressure then the next action is going to be down,
if it has strong upward pressure then the next action will be up. If the ration between
downward pressure and upward pressure isn't that big, then we don't know.

We should also consider the ration between the pressures and the actual candle.
If pressures are huge but the candle is small that means we're at an inflection point.



'''



'''

For RISING Trend
Factors which seem to suggest rising:
    1. Increase in range between open and close
    2. If the close and the high are close together

Factors which seem to suggest falling
    1. Decrease in range between open and close
    2. If the close and the high are far apart
    3. If the low is trailing down (the lows are more and more)

For FALLING Trend
Factors which seem to suggest rising:
    1. Increase in range between open and close
    2. If the high is trailing up

Factors which seem to suggest falling:
    1. Decrease in range between open and close
    2. If the low is trailing down
'''

'''
Hypothesis:
    We can use the candlestick chart to predict the immediate short term movement of stocks.
    How do we do that?

    Notice that right before the stock starts to go down, the low is fairly low, and right
        before the stock begins to go up, the high is fairly high, even if the close is not
        that high.

    Ok, how do we turn this into an algorithm. Basically:
    
    IF OPEN > CLOSE
    ELIF CLOSE > OPEN
        if low - open > close - open: we're going down
        elif open - close > high - close: we're going up

'''